Part 1

In [1]:
from re import finditer
from itertools import combinations

def is_valid(text):
    groups = [len(x.group()) for x in finditer(r'#+', text)]
    counts = [int(x.group()) for x in finditer(r'\d+', text)]
    return all(g == c for g, c in zip(groups, counts))

def permutate(line, combs):
    for comb in combs:
        test_line = line
        for i in comb:
            test_line = test_line[:i] + '#' + test_line[i+1:]
        if is_valid(test_line):
            yield test_line

def pt1(filename):
    with open(filename, 'r') as f:
        data = f.read().splitlines()

    perm_counts = 0
    for line in data:
        unknown_inds = [x.start() for x in finditer(r'\?', line)]
        counts = [int(x.group()) for x in finditer(r'\d+', line)]
        n_todo = sum(counts) - line.count('#')

        unknown_inds = [x.start() for x in finditer(r'\?', line)]
        combs = combinations(unknown_inds, n_todo)
        
        perms = permutate(line, combs)
        perm_counts += len([x for x in perms])

    return perm_counts

pt1('test.txt'), pt1('input.txt')

(21, 7260)

Part 2

In [2]:
def get_combos(springstr, groups, key, cache):
    if key in cache:
        return cache[key]
    
    i, grp_i, grp_len = key
    if i == len(springstr):
        return any((
            grp_i == len(groups)     and grp_len == 0,
            grp_i == len(groups) - 1 and grp_len == groups[grp_i]
        ))
    
    next_keys = [
        (i + 1, grp_i, 0),
        (i + 1, grp_i + 1, 0),
        (i + 1, grp_i, grp_len + 1)
    ]
    inclusion = [
        springstr[i] in '.?' and grp_len == 0,
        springstr[i] in '.?' and grp_i < len(groups) and groups[grp_i] == grp_len,
        springstr[i] in '#?'
    ]
    cache[key] = sum(
        get_combos(springstr, groups, k, cache)
        for include, k in zip(inclusion, next_keys) if include
    )
    return cache[key]


def pt2(filename, reps=1):
    with open(filename, 'r') as f:
        data = f.read().splitlines()

    combos = 0
    for line in data:
        springstr, groups = line.split()
        springstr = '?'.join([springstr] * reps)
        groups = [int(x) for x in (groups.split(',') * reps)]
        combos += get_combos(springstr, groups, (0, 0, 0), dict())
        
    return combos

pt2('test.txt'), pt2('input.txt'), pt2('test.txt', 5), pt2('input.txt', 5)

(21, 7260, 525152, 1909291258644)